## Perzeptron

In this notebook, we design, implement, train and try out a perceptron as a binary classifier for the given iris dataset.

The dataset is in this case a simplified version of the original dataset from the UCI.
While the original dataset has three target classes, this simplified version has only two.
The number of instances is the same. See https://archive.ics.uci.edu/ml/datasets/iris for the original.

* The four features `xi` are `x1` named `sepal_length`, `x2` named `sepal_width`, `x3` named `petal length`, `x4` named `petal width`.
* Together the features form the vector `x`.
* Note that the unit of measurement of width and length is in this case unknown, however also not relevant to the task at hand.

The dataset has one binary target.

* The target is `y` with its column name `class` is categorical and binary.
* Note that where we actually use the categorical data type of the library pandas for it, we call the column `class_cat`.

Our perceptron is going to have one weight per feature and the additive weiht b for the bias.

* We call the weights `wi` simply ```w1, w2, w3, w4```, each of which corresponds to the feature `xi` that has the same index `i`.
* Together these four weights form the vector `w`.
* The additive weight `b` represents the bias and is in this case stored outside of `w`.

Our perceptron yields binary output.

* The prediction obtained at the output of the perceptron we call `y_hat`.
* The difference between a known result `y` and a predicted result `y_hat` we call `y_diff`.
* Note that `y` and `y_hat` can also be vectors in case multiple results and corresponding predictions are referred to.

Our perceptron has a learning rate built in.

* The lerning rate alpha we call `lr`.

The perceptron has a model function.

* The function `perceptron_model_func` is the model function of the perceptron.
* It is a Heaviside function, i.e. a step function.
* It has two steps with a singularity just before 0.
* It accepts the vector `w` with all the non-additive weights.
* It accepts the vector `x` in order to make multiple predictions at once.
* It accepts `b` as a scalar.

The perceptron has another function to train it.

* The function `perceptron_train_func` trains the perceptron.
* It contains the part that updates the weights `w` and `b`.
* It accepts known data `x` and `y` which should be vectors, i.e. multiple known instances.
* It also requires initial weight values which can just be initialized randomly.
* Internally it uses the function `perceptron_model_func` to compute `y_hat` in each iteration. 


#### Imports & Settings

In [ ]:
# in this code block we gather all imports

import numpy as np

# MinMaxScaler for data normalization
from sklearn.preprocessing import MinMaxScaler

# for some plots and the scatter matrix
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm

# pandas for reading the CSV and for use with the library ppscore
import pandas as pd
from pandas.api.types import CategoricalDtype

# ppscore for exploratory data analysis
import ppscore as pps

# more statistics for exploratory data analysis
from scipy import stats

#### Load Data

In [ ]:
# in this code block we read the CSV file and perform some basic preprocessing

# read the CSV file
# using separator character semicolon
df_all_pd = pd.read_csv("../../data/iris_binary.csv", sep=',', skipinitialspace=True)

# make column names pythonic
# so that they can be used in code where applicable
df_all_pd.columns = df_all_pd.columns.str.replace(" ", "_")

# on a side note we choose to sort the data frame by the first column 
df_all_pd.sort_values(by='sepal_length', ascending=True, axis=0, inplace=True)

dataset_known = df_all_pd.to_numpy()

x_known_pd = df_all_pd.copy().drop('class', axis=1)
x_known = x_known_pd.to_numpy()

y_known_pd = df_all_pd['class'].copy()
y_known = y_known_pd.to_numpy()

n = dataset_known.shape[0]
assert dataset_known.shape == (n,5)
assert x_known.shape == (n,4)
assert y_known.shape == (n,)


In [ ]:
df_all_pd

#### Exploratory Data Analysis

In [ ]:
# in this code block we do an EDA

# number of instances
print(f"n={n}")

# location parameters
print(f"mean={dataset_known.mean(axis=0)}")
print(f"trimmed_mean={stats.trim_mean(dataset_known.astype('float32'), proportiontocut=0.10, axis=0)}")
print(f"mode={stats.mode(dataset_known, keepdims=True)}")

# statistical dispersion measures
def range_np(a: np.ndarray) -> np.ndarray:
    result = a.max(axis=0) - a.min(axis=0)
    return result

print(f"range={range_np(dataset_known)}")
print(f"iqr={stats.iqr(dataset_known, axis=0)}")

print(f"percentile_10={np.percentile(dataset_known, 10.0, axis=0)}")
print(f"percentile_25={np.percentile(dataset_known, 25.0, axis=0)}")
print(f"median={np.percentile(dataset_known, 50.0, axis=0)}")
print(f"percentile_75={np.percentile(dataset_known, 75.0, axis=0)}")
print(f"percentile_90={np.percentile(dataset_known, 90.0, axis=0)}")

def mad_np(a: np.ndarray) -> np.ndarray:
    result = np.mean(np.absolute(a - np.mean(a, axis=0)), axis=0)
    return result

print(f"mad={mad_np(dataset_known)}")

print(f"std={dataset_known.std(axis=0)}")
print(f"var={dataset_known.var(axis=0)}")

# association measures
print(f"\ncorrelation_matrix=\n{np.corrcoef(dataset_known, rowvar=False).round(decimals=2)}")

# we have a look at a scatter matrix
pd.plotting.scatter_matrix(df_all_pd, 
                           c=df_all_pd['class'], 
                           figsize=(17, 17),
                           cmap = cm['cool'],
                           diagonal = 'kde')

# for the computation of predictive power scores we use pandas categorical data type for class_cat
class_categories_pd = CategoricalDtype(categories=[0, 1], ordered=True)
df_all_with_class_cat_pd = df_all_pd.copy()
df_all_with_class_cat_pd['class_cat'] = df_all_pd['class'].astype(class_categories_pd)
df_all_with_class_cat_pd.drop('class', axis=1)

#predictive_power_score_matrix_all_pd = pps.matrix(df_pd_all, output='df')
predictive_power_scores_pd = pps.predictors(df_all_with_class_cat_pd, y='class_cat', output='df')
predictive_power_scores_pd.style.background_gradient(cmap='twilight', low=0.0, high=1.0)


#### Initial Weights

In [ ]:
# in this code block we initialize the vector w and the bias b

#w_init = np.asarray([0.5, 1.0, -0.5, -1.0])
w_init = np.asarray([-0.0, 1.0, 1.0, 1.0])
b_init = 0.0005



#### Model Function

In [ ]:
# in this code block we define the model function

def perceptron_model_func(w, x, b):
    # thanks to the use of np.matmul(x, w)
    # w can be the vector w with all weights
    # x can be a matrix of many instances with len(w) columns and n rows
    # b can be a scalar or a numpy array of shape (1,)
    assert w.shape == (4,)
    assert x.shape == (150, w.shape[0])
    assert np.isscalar(b) or b.shape == (1,)

    y_pure = np.matmul(x, w).round(4) + b

    assert not np.isscalar(y_pure)
    assert y_pure.dtype == 'float64'
    assert y_pure.shape == (x.shape[0],)
    
    y_hat = np.heaviside(y_pure, 1)

    assert not np.isscalar(y_hat)
    assert y_hat.dtype == 'float64'
    assert y_hat.shape == (x.shape[0],)
    return y_hat

# try out the model function once with a twist and some assertions

# for this example we use a b that will be recognizable in the y_pure values
b_init_example_1 = 0.0005
# and we use a w1 equals zero to effectively deactivate that feature
w_init_example_1 = np.asarray([-0.0, 1.0, 1.0, 1.0])

y_pure_example_1 = np.matmul(x_known, w_init_example_1).round(4) + b_init
assert not np.isscalar(y_pure_example_1)
assert y_pure_example_1.dtype == 'float64'
assert y_pure_example_1.shape == (150,)

y_hat_example_1 = perceptron_model_func(w_init_example_1, x_known, b_init_example_1)
assert not np.isscalar(y_hat_example_1)
assert y_hat_example_1.dtype == 'float64'
assert y_hat_example_1.shape == (150,)

for j in range(n):
    #print(f"x{j}={x_known[j]}")
    #print(f"y_pure_example_1{j}={y_pure_example_1[j]}")
    # we use the especially chosen b_init_example_1 and w_init_example_1
    # to be able to do the following assertion
    y_pure_example_1_expected = x_known[j,1:].sum().round(2) + 0.0005
    #print(f"y_pure_example_1_expected={y_pure_example_1_expected}")
    assert y_pure_example_1_expected == y_pure_example_1[j]


In [ ]:
# in this code block we define the more simple model function needed during training

# the difference to the main model function above is that the simple version
# only works for one instance x_j at a time

def perceptron_model_func_for_train(w_j, x_j, b_j):
    assert w_j.shape == (4,)
    assert x_j.shape == w_j.shape
    assert np.isscalar(b_j) or b_j.shape == (1,)
    y_hat_pure_j = np.dot(w_j, x_j) + b_j
    y_hat_j = np.heaviside(y_hat_pure_j, 1)
    return y_hat_j


#### Weight Update Functions

In [ ]:
# in this code block we experiment with the implementation for learning with respect to vector w

# note that the function in this code block is experimental
# since it computes all learning steps in one operation based on numpy broadcasting
# which however does not meet the requirements of the perceptron learning approach
# thus for the actual function to update the weights see next code block

# formula is wi_new = wi + lr * (y - y_hat) * xij
# index i is the index of the feature
# index j is the index of the known instance

def perceptron_train_func_experimental(w, lr, y, y_hat, x):
    # w can be the vector w with all weights
    # x can be a matrix of many instances with len(w) columns and n rows
    # b can be a scalar or a numpy array of shape (1,)
    
    # number of features i.e. number of weights i.e. number of columns
    m_local = w.shape[0]

    # number of given instances i.e. rows
    n_local = y.shape[0]

    # check the shapes of the vectors w, y, y_hat
    assert w.shape == (m_local,)
    assert y.shape == (n_local,)
    assert y_hat.shape == (n_local,)
    # check the shape of the matrix x
    assert x.shape == (n_local, m_local)

    y_diff = y - y_hat
    assert y_diff.shape == (n_local,)

    # making use of the numpy broadcasting rules
    # we add an empty dimension to y_diff to prepare for the following multiplication
    y_diff_expanded = np.expand_dims(y_diff, axis=1)
    assert y_diff_expanded.shape == (n_local, 1)

    y_diff_x = y_diff_expanded * x
    assert y_diff_x.shape == (n_local, m_local)

    lr_y_diff_x = lr * y_diff_x

    w_new = w + lr_y_diff_x
    
    # this printing was just for debugging purposes
    #print(f"w            {w}")
    #for j in range(n_local):
    #    print(f"x            {x[j]}")
    #    print(f"y            {y[j]}")
    #    print(f"y_hat        {y_hat[j]}")
    #    print(f"y_diff       {y_diff_expanded[j]}")
    #    print(f"y_diff_x     {y_diff_x[j]}")
    #    print(f"lr_y_diff_x  {lr_y_diff_x[j]}")
    #    print(f"w_new        {w_new[j]}")

    assert w_new.shape == (n_local, m_local)
    return w_new

# try out the function
w_new_experimental = perceptron_train_func_experimental(w_init_example_1, 0.01, y_known, y_hat_example_1, x_known)
#print(w_new_experimental)

In [ ]:
# in this code block we implement the function to update the weights

def perceptron_train_func(w, lr, y, x, b, debug):
    # w can be the vector w with all weights
    # x can be a matrix of many instances with len(w) columns and n rows
    # b can be a scalar or a numpy array of shape (1,)
    
    # number of features i.e. number of weights i.e. number of columns
    m_local = w.shape[0]

    # number of given instances i.e. rows
    n_local = y.shape[0]

    # check the shapes of the vectors w, y, y_hat
    assert w.shape == (m_local,)
    assert y.shape == (n_local,)    
    # check the shape of the matrix x
    assert x.shape == (n_local, m_local)

    w_j = w
    b_j = b
    for j in range(n_local):
        # for each instance
        x_j = x[j]
        assert x_j.shape == (m_local,)

        y_j = y[j]
        assert np.isscalar(y_j)

        y_hat_j = perceptron_model_func_for_train(w_j, x_j, b)
        assert np.isscalar(y_hat_j)

        y_diff_j = y_j - y_hat_j
        assert np.isscalar(y_diff_j)

        w_change_j = lr * (y_diff_j * x_j)
        w_new_j = w_j + w_change_j

        b_change_j = lr * (y_diff_j)
        b_new_j = b_j + b_change_j

        if debug:
            print(f"w_j            {w_j}")
            print(f"x_j            {x_j}")
            print(f"y_j            {y_j}")
            print(f"y_hat_j        {y_hat_j}")
            print(f"y_diff_j       {y_diff_j}")
            print(f"change_j       {w_change_j}")
            print(f"w_new_j        {w_new_j}")
            print(f"b_change_j     {b_change_j}")
            print(f"b_new_j        {b_new_j}")
            print(f"----------------------------------------------------------------")
        w_j = w_new_j
        b_j = b_new_j

    return (w_j, b_j)

# try out the function
debug = False
w_new_example_1, b_new_example_1 = perceptron_train_func(w_init_example_1, 0.01, y_known, x_known, b_init_example_1, debug)
print(w_new_example_1)
print(b_new_example_1)


In [ ]:
# in this code block we first train the perceptron and then use it

# train
w_trained, b_trained = perceptron_train_func(w_init, 0.01, y_known, x_known, b_init, False)

print(f"w_trained={w_trained}")
print(f"b_trained={b_trained}")

# apply model function based on matmul
y_hat_example_2 = perceptron_model_func(w_trained, x_known, b_trained)

# apply naive model function
y_hat_example_2_simple = np.zeros(n)
for j in range(n):
    y_hat_example_2_simple[j] = perceptron_model_func_for_train(w_trained, x_known[j], b_trained)

# check the results
print(y_known)
y_hat_example_2_int = y_hat_example_2.astype('int32')
y_hat_example_2_simple_int = y_hat_example_2_simple.astype('int')

# mask where values are not equal which should not be the case
y_hat_example_2_int_masked = np.ma.masked_where((y_known != y_hat_example_2_int.copy()).all(), y_hat_example_2_int)
print(y_hat_example_2_int_masked)
assert y_hat_example_2_int_masked.compressed().size == n
# mask where values are equal which should everywhere be the case
y_hat_example_2_int_masked_negative = np.ma.masked_where((y_known == y_hat_example_2_int.copy()).all(), y_hat_example_2_int)
print(y_hat_example_2_int_masked_negative)
print(y_hat_example_2_int_masked_negative.compressed())
assert y_hat_example_2_int_masked_negative.compressed().size == 0

# mask where values are not equal which should not be the case
y_hat_example_2_simple_int_masked = np.ma.masked_where((y_known != y_hat_example_2_simple_int.copy()).all(), y_hat_example_2_simple_int)
print(y_hat_example_2_simple_int_masked)
assert y_hat_example_2_simple_int_masked.compressed().size == n
# mask where values are equal which should everywhere be the case
y_hat_example_2_simple_int_masked_negative = np.ma.masked_where((y_known == y_hat_example_2_simple_int.copy()).all(), y_hat_example_2_simple_int)
print(y_hat_example_2_simple_int_masked_negative)
assert y_hat_example_2_simple_int_masked_negative.compressed().size == 0

assert (y_known == y_hat_example_2_int).all()
assert (y_known == y_hat_example_2_simple_int).all()
